# PyTorch模型ONNX导出和推理示例 （源于 Pytorch 官方教程）

本notebook展示了如何将PyTorch模型导出为ONNX格式，并使用ONNX Runtime进行推理。主要包含以下步骤：

1. 加载和初始化PyTorch模型
2. 导出模型为ONNX格式
3. 验证ONNX模型
4. 使用ONNX Runtime进行推理
5. 性能对比测试
6. 实际图像处理示例

## 1. 模型初始化

首先导入必要的库并初始化测试模型。这里使用的是自定义的test_network模型，输入通道数为3，输出类别数为10。

In [ ]:
# 导入必要的库
import torch
from models.test_model2 import test_network

# 设置批次大小
batch_size = 16

# 创建测试网络实例：3个输入通道，10个输出类别
model = test_network(3, 10)
# 设置为评估模式，关闭dropout和batch normalization
model.eval()

test_network(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (global_avg_pool): AdaptiveAvgPool2d(output_size=1)
  (fc): Linear(in_features=128, out_features=10, bias=True)
)

## 2. 导出ONNX模型

将PyTorch模型导出为ONNX格式。ONNX（Open Neural Network Exchange）是一个开放的深度学习模型交换格式，支持跨平台和跨框架的模型部署。

In [27]:
# 创建模拟输入数据：批次大小16，3通道，224x224分辨率
x = torch.randn(16, 3, 224, 224, requires_grad=True)
# 使用PyTorch模型进行前向推理，获取输出作为参考
torch_out = model(x)

# 导出PyTorch模型为ONNX格式
torch.onnx.export(
    model,  # 要导出的PyTorch模型
    x,  # 模型输入（用于追踪计算图）
    "../temp_data/test.onnx",  # 保存路径
    export_params=True,  # 是否导出模型参数权重
    do_constant_folding=True,  # 是否执行常量折叠优化
    input_names=["input"],  # 输入节点名称
    output_names=["output"],  # 输出节点名称
    dynamic_axes={
        "input": {0: "batch_size"},  # 动态轴：支持可变批次大小
        "output": {0: "batch_size"},
    },
)


/tmp/ipykernel_2986956/2290931629.py:7: DeprecationWarning: You are using the legacy TorchScript-based ONNX export. Starting in PyTorch 2.9, the new torch.export-based ONNX exporter will be the default. To switch now, set dynamo=True in torch.onnx.export. This new exporter supports features like exporting LLMs with DynamicCache. We encourage you to try it and share feedback to help improve the experience. Learn more about the new export logic: https://pytorch.org/docs/stable/onnx_dynamo.html. For exporting control flow: https://pytorch.org/tutorials/beginner/onnx/export_control_flow_model_to_onnx_tutorial.html.
  torch.onnx.export(


## 3. 验证ONNX模型

加载导出的ONNX模型并验证其结构是否正确。这是确保模型导出成功的重要步骤。

In [17]:
# 导入ONNX库
import onnx

# 加载导出的ONNX模型
onnx_model = onnx.load("super_resolution.onnx")
# 检查模型结构是否有效
onnx.checker.check_model(onnx_model)


## 4. ONNX Runtime推理测试

使用ONNX Runtime加载模型并进行推理，然后与原始PyTorch模型的输出进行对比，确保转换的准确性。

In [18]:
# 导入ONNX Runtime和NumPy
import onnxruntime
import numpy as np

# 创建ONNX Runtime推理会话，使用CPU执行提供程序
ort_session = onnxruntime.InferenceSession(
    "super_resolution.onnx", providers=["CPUExecutionProvider"]
)


def to_numpy(tensor):
    """将PyTorch张量转换为NumPy数组的辅助函数"""
    return (
        tensor.detach().cpu().numpy() if tensor.requires_grad else tensor.cpu().numpy()
    )


# 使用ONNX Runtime进行推理
ort_inputs = {ort_session.get_inputs()[0].name: to_numpy(x)}
ort_outs = ort_session.run(None, ort_inputs)

# 比较ONNX Runtime和PyTorch的输出结果
# 使用较小的容差来验证数值精度
np.testing.assert_allclose(to_numpy(torch_out), ort_outs[0], rtol=1e-03, atol=1e-05)

print("ONNX模型已通过ONNXRuntime测试，结果与PyTorch模型一致！")


ONNX模型已通过ONNXRuntime测试，结果与PyTorch模型一致！


## 5. 性能对比测试

比较PyTorch模型和ONNX模型的推理速度，通常ONNX Runtime在某些场景下可以提供更好的性能。

In [23]:
# 导入时间模块用于性能测试
import time

# 创建新的测试输入数据
x = torch.randn(batch_size, 3, 224, 224, requires_grad=True)

# 测试PyTorch模型推理时间
start = time.time()
torch_out = model(x)
end = time.time()
print(f"PyTorch模型推理耗时: {end - start:.6f} 秒")

# 测试ONNX模型推理时间
ort_inputs = {ort_session.get_inputs()[0].name: to_numpy(x)}
start = time.time()
ort_outs = ort_session.run(None, ort_inputs)
end = time.time()
print(f"ONNX模型推理耗时: {end - start:.6f} 秒")


PyTorch模型推理耗时: 0.039594 秒
ONNX模型推理耗时: 0.011369 秒


## 6. 实际图像处理示例

使用导出的ONNX模型处理实际图像。这个示例展示了如何使用加载硬盘中的图像并输出对应的概率。

**处理流程：**
1. 加载图像并调整大小
2. 将硬盘中的数据输入到网络中
3. 输出网络结果

In [26]:
# 导入图像处理库
from PIL import Image
import torchvision.transforms as transforms

# 加载测试图像
img = Image.open("../temp_data/cat.png")

# 调整图像大小到224x224（与模型输入尺寸匹配）
resize = transforms.Resize([224, 224])
img = resize(img)

# 将图像转换为张量格式
to_tensor = transforms.ToTensor()
img = to_tensor(img)
img.unsqueeze_(0)  # 添加批次维度

# 使用ONNX模型对图像进行超分辨率处理
ort_inputs = {ort_session.get_inputs()[0].name: to_numpy(img)}
ort_outs = ort_session.run(None, ort_inputs)
img_out = ort_outs[0]

# 输出图像在 ONNX 的输出
print("输出图像形状:", img_out.shape)

输出图像形状: (1, 10)


## 总结

本notebook成功演示了PyTorch模型到ONNX的完整转换流程：

### 主要步骤
1. **模型准备**: 加载并设置PyTorch模型为评估模式
2. **ONNX导出**: 使用`torch.onnx.export()`将模型导出为ONNX格式
3. **模型验证**: 使用ONNX库验证导出模型的正确性
4. **推理测试**: 使用ONNX Runtime进行推理并与原模型对比
5. **性能评估**: 比较两种推理方式的执行时间
6. **实际应用**: 在真实图像上测试分类效果

### 关键优势
- **跨平台部署**: ONNX格式支持多种推理引擎和硬件平台
- **性能优化**: ONNX Runtime通常提供更好的推理性能
- **模型兼容性**: 保持与原始PyTorch模型相同的推理结果

### 输出文件
- `../temp_data/test.onnx`: 导出的ONNX模型文件